In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time

In [5]:
# Station and wharves dataframe
wharf_df = pd.read_csv('csv_inputs/wharf_info.csv')
# lines dataframe
line_df = pd.read_csv('csv_inputs/line_info.csv')
line_df['First_sailing'] = pd.to_datetime(line_df['First_sailing'], format='%H:%M')
# Wharf to wharf transit time dataframe
tt_df = pd.read_csv('csv_inputs/rebalancing_times.csv',index_col='From/To')
# Headways dataframe
headway_df = pd.read_csv('csv_inputs/headways.csv')
# vessels
vessel_df = pd.read_csv('csv_inputs/vessel_info.csv')
# charging berths dataframe
charging_berth = pd.read_csv('csv_inputs/charging_berths.csv')

initial_time = time(5,0)
period_length = 5 # mins
Dc = 60  # Crew break duration in minutes


# Functions

In [6]:
def cal_time(period_num):
    """
    Calculates a new datetime based on an initial time and a given number of periods.
    
    Args:
    period_num (int): The period number to calculate the time for.
    initial_time (datetime.time): The starting time of the calculation.
    period_length (int): The length of each period in minutes.
    
    Returns:
    datetime.datetime: The calculated datetime.
    """
    # Convert initial_time to a datetime object with today's date
    initial_datetime = datetime.combine(datetime.today(), initial_time)
    
    # Calculate the total minutes to add
    added_time = timedelta(minutes=int((period_num - 1) * period_length))
    new_time = initial_datetime + added_time

    return new_time


In [7]:
def cal_h(s, d, line): # copy from previous file
    """
    Calculate the s-th sailing time starting from time 'd' for a specified 'line'.

    This function retrieves headway periods for a specific line from a dataframe,
    constructs a list of sailing times starting from time 'd', and returns the s-th
    sailing time based on these intervals. It is used to project sailing schedules
    based on a specified headway and start day.

    Parameters:
    s (int): The order of the sailing time to be retrieved (1st, 2nd, etc.).
    d (int): The day from which sailing starts.
    line (int): The line number for which headway data is to be used.

    Returns:
    int or None: The s-th sailing time in terms of the number of time periods
                 since day 'd', or None if the index 's' is out of range or
                 an error occurs in processing.

    Note:
    The function assumes `period_length` as a global variable that denotes the
    length of each time period within the operational schedule.
    """

    try:
        # Retrieve the headway periods for the specified line and drop missing values
        h = headway_df[f'h{line}'].dropna().tolist()
        h_sd_ls = [d]  # Start the list with the initial day 'd'

        # Calculate subsequent sailing times based on headway periods
        for sailing_headway in h:
            num_time_period = int(sailing_headway // period_length + 1)  # round up
            h_sd_ls.append(h_sd_ls[-1] + num_time_period)

        # Return the s-th sailing time if within bounds
        if s-1 < len(h_sd_ls):
            return h_sd_ls[s-1]
        else:
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [8]:
# Functions to replace and determine wharfs
def replace_wharf(wharf):
    if wharf.isdigit():
        return Start_wharf.get(wharf, wharf)
    return wharf

def end_wharf(row):
    wharf = row['End_Wharf']
    if row['Task'].isdigit():
        line = int(row['Task'])
        timetoT = int(line_df[line_df['Line_No'] == line]['Time_underway_to_T'].iloc[0])
        end_time = row['Start_Time'] + timetoT // period_length + 1
        matching_rows = Zp_df[(Zp_df['Line'] == line) & (Zp_df['Time'] == end_time)]
        if not matching_rows.empty:
            wharf = matching_rows['Wharf'].iloc[0]
    return wharf

def end_time(row):
    if row['Task'].isdigit():
        line = int(row['Task'])
        duration = int(line_df[line_df['Line_No'] == line]['Line_duration'].iloc[0])
        return (cal_time(row['Start_Time']) + timedelta(minutes=duration)).strftime('%H:%M')
    elif row['Task'] in ['Waiting', 'Charging']:
        return (cal_time(row['Start_Time']) + timedelta(minutes=5)).strftime('%H:%M')
    else:
        return (cal_time(row['Start_Time']) + timedelta(minutes=Dc)).strftime('%H:%M')

# Load results files

In [11]:
def load_and_process_data(filepath, split_columns, value_columns):
    """
    Load data, remove unwanted characters, filter rows, split columns, and convert types.

    Args:
    filepath (str): Path to the CSV file.
    split_columns (list): List of columns names after splitting the 'Variable' column.
    value_columns (dict): Dictionary of columns to convert with their respective types.
    """
    df = pd.read_csv(filepath)
    df = df[df['Value'] == 1]
    df['Variable'] = df['Variable'].str.replace(r"[()']", "", regex=True)
    df[split_columns] = df['Variable'].str.split(',', expand=True)
    for column, dtype in value_columns.items():
        df[column] = df[column].astype(dtype)
    return df

# Load and process each DataFrame using the new function
z_df = load_and_process_data('Output_files/z_wj_results.csv',['Wharf', 'Line'],{'Line': int})

Zp_df = load_and_process_data('Output_files/Z_prime_lwt_results.csv',['Line', 'Wharf', 'Time'],{'Line': int, 'Time': int})

Z_df = load_and_process_data('Output_files/Z_lwt_results.csv',['Line', 'Wharf', 'Time'],{'Line': int, 'Time': int})



In [13]:
# start stations from line and wharf data
Start_S = dict(zip(line_df['Line_No'].astype(str), line_df['O']))
Start_S.update(dict(zip(wharf_df['Wharf_No'], wharf_df['Station'])))

# end stations from line and wharf data
End_S = dict(zip(line_df['Line_No'].astype(str), line_df['T']))
End_S.update(dict(zip(wharf_df['Wharf_No'], wharf_df['Station'])))

# start wharfs from z_df
Start_wharf = dict(zip(z_df['Line'].astype(str), z_df['Wharf']))

# end wharfs from Zp_df will be determined later

# Timetable

In [14]:
# Load and preprocess x_df
x_df = load_and_process_data('Output_files/x_ld_results.csv',['Line', 'Time'],{'Line': int, 'Time': int})


# List of lines from line_df and x_df
lines = line_df['Line_No'].unique().tolist()
solved_lines = x_df['Line'].unique().tolist()


def cal_timetable(line):
    # Check if the line exists and is solved
    if line in lines and line in solved_lines:
        # First sailing time and headways
        d = x_df.loc[x_df['Line'] == line, 'Time'].iloc[0]
        headways = headway_df[f'h{line}'].dropna().tolist()

        times, locs, wharfs = [], [], []
        
        # Calculate timings for all sailings including first and subsequent ones
        for s in range(len(headways)+1):
            period = cal_h(s+1, d, line)  # Assuming cal_h is defined somewhere
            sailing_time = cal_time(period)
            
            # Add sailing time, location and wharf
            times.append(sailing_time)
            locs.append(Start_S[str(line)])
            wharfs.append(z_df[z_df['Line'] == line]['Wharf'].iloc[0])

            # Intermediate Stop
            Intermediate_stop = line_df[line_df['Line_No'] == line]['I'].iloc[0]
            if pd.notna(Intermediate_stop):
                timetoI = int(line_df[line_df['Line_No'] == line]['Time_underway_to_I'].iloc[0])
                arrival_time_I = sailing_time + timedelta(minutes=timetoI)
                times.append(arrival_time_I)
                locs.append(Intermediate_stop)
                wharfs.append(z_df[z_df['Line'] == line]['Wharf'].iloc[1])
            
            # Terminal Stop
            timetoT = int(line_df[line_df['Line_No'] == line]['Time_underway_to_T'].iloc[0])
            arrival_time_T = sailing_time + timedelta(minutes=timetoT)
            times.append(arrival_time_T)
            locs.append(End_S[str(line)])
            wharfs.append(Zp_df[(Zp_df['Line'] == line) & (Zp_df['Time'] == period + timetoT // period_length + 1)]['Wharf'].iloc[0])

        # Format times and create the DataFrame
        formatted_times = [time.strftime('%H:%M') for time in times]
        timetable = pd.DataFrame({
            'Time': formatted_times,
            'Station': locs,
            'Wharf': wharfs
        })
        return timetable
    
    else:
        print('Line not exist or unsolved.')
        return None

cal_timetable(1)

,Time,Station,Wharf
0,06:30,Circular Quay,CQ2
1,06:45,Mosman,Mos1
2,07:05,Circular Quay,CQ2
3,07:20,Mosman,Mos1
4,07:40,Circular Quay,CQ2
5,07:55,Mosman,Mos1
6,08:15,Circular Quay,CQ2
7,08:30,Mosman,Mos1


In [15]:
cal_timetable(5)

,Time,Station,Wharf
0,06:50,Circular Quay,CQ5
1,07:01,Barangaroo,Bar5
2,08:16,Parramatta,Pm1
3,07:15,Circular Quay,CQ5
4,07:26,Barangaroo,Bar5
...,...,...,...
136,01:46,Barangaroo,Bar5
137,03:01,Parramatta,Pm1
138,02:00,Circular Quay,CQ5
139,02:11,Barangaroo,Bar5


# Vessel itinerary

In [16]:
# Load data
y_df = load_and_process_data('Output_files/y_vjt_results.csv',['Vessel', 'Task', 'Start_Time'],{'Start_Time': int})

# Extract and clean up Wharf details
y_df['Start_Wharf'] = y_df['Task'].apply(lambda x: x.split('_')[-1].strip())
y_df['End_Wharf'] = y_df['Start_Wharf']

# Lookup Start and End Stations based on Wharfs
y_df['Start_Station'] = y_df['Start_Wharf'].apply(lambda x: Start_S.get(x, 'Unknown Station'))
y_df['End_Station'] = y_df['Start_Wharf'].apply(lambda x: End_S.get(x, 'Unknown Station'))

# Update Task based on specific keywords or conditions
y_df['Task'] = y_df['Task'].apply(
    lambda x: 'Charging' if x.strip().lower().startswith('phi') else
              'Crew pause' if x.strip().lower().startswith('cp') else
              x.strip() if x.strip().isdigit() else
              'Waiting'
)


# Apply transformations and update columns
y_df['Start_Wharf'] = y_df['Start_Wharf'].apply(replace_wharf)
y_df['End_Wharf'] = y_df.apply(end_wharf, axis=1)
y_df['End_Time'] = y_df.apply(end_time, axis=1)
# y_df['Start_Time'] = y_df['Start_Time'].apply(lambda x: cal_time(x).strftime('%H:%M')) # DONOT CONVERT THE TIME NOW

# Reorganize DataFrame columns for final output
y_df = y_df[['Vessel', 'Task', 'Start_Station', 'Start_Wharf', 'Start_Time', 'End_Station', 'End_Wharf', 'End_Time']]

# Function to calculate vessel itinerary
def cal_itinerary(vessel):
    itinerary = y_df[y_df['Vessel'] == vessel].sort_values('Start_Time')
    itinerary['Start_Time'] = itinerary['Start_Time'].apply(lambda x: cal_time(x).strftime('%H:%M'))
    itinerary.reset_index(inplace=True, drop=True)
    return itinerary

In [17]:
cal_itinerary('EC3')

,Vessel,Task,Start_Station,Start_Wharf,Start_Time,End_Station,End_Wharf,End_Time
0,EC3,Waiting,Balmain Shipyard,BSY5,05:00,Balmain Shipyard,BSY5,05:05
1,EC3,Waiting,Balmain Shipyard,BSY6,05:05,Balmain Shipyard,BSY6,05:10
2,EC3,Waiting,Balmain Shipyard,BSY6,05:10,Balmain Shipyard,BSY6,05:15
3,EC3,Waiting,Barangaroo,Bar4,05:15,Barangaroo,Bar4,05:20
4,EC3,Waiting,Balmain Shipyard,BSY3,05:20,Balmain Shipyard,BSY3,05:25
5,EC3,Waiting,Balmain Shipyard,BSY3,05:25,Balmain Shipyard,BSY3,05:30
6,EC3,Charging,Circular Quay,CQ5,05:50,Circular Quay,CQ5,05:55
7,EC3,14,Circular Quay,CQ4,06:00,Cockatoo Island,CI1,06:38
8,EC3,16,Circular Quay,CQ2,07:15,Watsons Bay,WB1,07:49
9,EC3,15,Cockatoo Island,CI1,08:40,Circular Quay,CQ2,09:17


In [18]:
cal_itinerary('10M')

,Vessel,Task,Start_Station,Start_Wharf,Start_Time,End_Station,End_Wharf,End_Time
0,10M,Waiting,Barangaroo,Bar5,05:00,Barangaroo,Bar5,05:05
1,10M,Waiting,Barangaroo,Bar5,05:05,Barangaroo,Bar5,05:10
2,10M,Waiting,Circular Quay,CQ3,05:10,Circular Quay,CQ3,05:15
3,10M,Waiting,Barangaroo,Bar4,05:15,Barangaroo,Bar4,05:20
4,10M,Charging,Watsons Bay,WB1,05:20,Watsons Bay,WB1,05:25
5,10M,Charging,Barangaroo,Bar1,05:50,Barangaroo,Bar1,05:55
6,10M,18,Barangaroo,Bar1,06:45,Blackwattle Bay,BWB1,07:04
7,10M,18,Barangaroo,Bar1,07:20,Blackwattle Bay,BWB1,07:39
8,10M,18,Barangaroo,Bar1,07:55,Blackwattle Bay,BWB1,08:14
9,10M,18,Barangaroo,Bar1,08:30,Blackwattle Bay,BWB1,08:49


# Wharfs utilisations
